# Pre-trained Language Models: SubTask C
## [8 Marks]

In this assignment, you will work on the [ComVE](https://competitions.codalab.org/competitions/21080) shared task that was part of SemEval-2020. The task aims to evaluate whether a system can distinguish if a natural language statement makes sense to humans or not and provide a reason. **ConVE** includes three subtasks that require models to acquire and apply commonsense knowledge. In this notebook you will focus on **SubTask C**:

- Given a statement that does not make sense, generate the reason why this statement does not make sense. For each nonsensical statement, three valid reasons are given as reference:

     *Statement*: He put an elephant into the fridge.  
     *Reason A*: An elephant is much bigger than a fridge.  
     *Reason B*: A fridge is much smaller than an elephant.  
     *Reason C*: Most of the fridges aren't large enough to contain an elephant.

     This subtask can be approached as a Sequence-to-Sequence problem where the input is the nonsensical statement and the output is a valid reason.

You will fine-tune a Pre-trained Language Model with [Transformers](https://huggingface.co/docs/transformers/index) library that provides a set of tools for fine-tunning and deploying a wide variety of Pre-trained Language Models. The [Hugging Face Hub](https://huggingface.co/models) allows you to explore all the models supported by **Transformers** and even share your own models with the community. In this assignment, you will work with [BART](https://huggingface.co/docs/transformers/model_doc/bart), a pre-trained Sequence-to-Sequence model.

Fine-tuning a Pre-trained Language Model usually requires a great amount of time and computational resources. Your personal computer will not be enough. In order to complete the assignment, you can work with a reduced version of the dataset and the base version of **BART**:

In [4]:
!pip install -r '/content/sample_data/Assessment+4_+requirements.txt'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 17.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1

In [5]:
shrink_dataset = True
base_model = True
colab = False

Although the value of these variables do not affect the tests that will evaluate your code, the output examples distributed throughout this notebook are based on a `shrink_dataset` and a `base_model` variables set as `True`, and a `colab` variable set as `False`.

If you want to perform a full training of the model to obtain its real performance, you can use a cloud service like [Google Colab](https://colab.research.google.com/). **Colab** is a **Jupyter** notebook environment that supports both GPU and TPU instances, allowing training large scale Deep Learning models. Set the `shrink_dataset` and a `base_model` variables to `False`, the `colab` variable to `True`, and follow the instructions provided to you to run the notebook in **Colab**.

In [6]:
if colab:
    ! pip install transformers datasets evaluate
    import os
    if not os.path.exists("SemEval2020-Task4-Data/ALL data/Training  Data/subtaskC_data_all.csv"):
        ! git clone https://github.com/wangcunxiang/SemEval2020-Task4-Commonsense-Validation-and-Explanation.git SemEval2020-Task4-Data

You will use the following objects and functions:

In [7]:
import pandas as pd
import evaluate
from datasets import Dataset
from transformers import (AutoTokenizer, AutoModelForSeq2SeqLM,
                          Seq2SeqTrainingArguments, Seq2SeqTrainer,
                          DataCollatorForSeq2Seq, enable_full_determinism)

When working with Neural Networks, there are a large number of random operations such as initializing the weights of the network, shuffling the data for training, or choosing samples. This causes that different training runs of the same model can lead to different results. To ensure reproducibility, i.e. obtaining the same results in the different runs, the random number generator must be initialized with a fixed value known as seed. In Transformers, this can be done as follows:

In [8]:
enable_full_determinism(seed=42)

> **Note!** With models as complex as Neural Networks, reproducibility is susceptible to factors such as software versions or the hardware on which the models are run. Even with seed initialization, there may be slight differences in the results.

Working with Neural Networks also involves defining a number of hyperparameters that set the configuration of the model. Finding the appropriate hyperparameter values requires training the model with different combinations and testing them on the development set. This hyperparameter tuning is a costly process that needs multiple rounds of experimentation. However, for this assignments, you will use the following values:

In [9]:
epochs = 3  # Number of epochs to train the model
train_batch_size = 8  # Number of examples used per gradient update
learning_rate = 1e-5  # The learning rate for the optimizer
max_length = 25  # Maximum lenght of the input sequence
output_dir = "modelC"  # The output directory where the model will be written

> **Note!** The notebook for this assignment provides very little guidance. You are expected to refer to the [documentation](https://huggingface.co/docs) for details on how to solve the exercises.

## Loading the Pre-trained Model - [1 Mark]

The first step you must perform in this assignment is to load the model and its corresponding tokenizer using the classes imported above.

In [10]:
def load_model(model_name):   #[1 Mark]

    # Load the tokenizer using the model name
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Load the model for sequence-to-sequence
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    return model, tokenizer

In [11]:
model_name = "facebook/bart-base" if base_model else "facebook/bart-large"
model, tokenizer = load_model(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

Some weights of BartForConditionalGeneration were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Data Pre-processing - [2 Marks]

The **ComVE** dataset consists of 10000 nonsensical statements for the train set, 997 statements for development and 1000 for test. Each nonsensical statements comes with with three reference valid reasons. You must load the three sets into three `DataFrames`. For the training and development splits, the `DataFrame` should contain three columns: the `id` of the nonsensical statement, a `FalseSent` column with the nonsensical statement and a `reason` column with the reference reasons. For the test set, the `DataFrame` should contain five columns: the `id` of the nonsensical statement, a `FalseSent` column with the nonsensical statement and three columns (`reason1`, `reason2` and `reason3`) containing each of the reference reasons.

Train DataFrame:

|       |   id | FalseSent                                         | reason                                                                         |
|------:|-----:|:--------------------------------------------------|:-------------------------------------------------------------------------------|
|   769 |  769 | Computers is an ingredient used in preparing food | Computers are not used for food and they are not edible                        |
| 10769 |  769 | Computers is an ingredient used in preparing food | Computer is not something that can be used in preparing food.                  |
| 20769 |  769 | Computers is an ingredient used in preparing food | You cannot eat a computer                                                      |
|   888 |  888 | he did hear music in his cooling glass            | cooling glass can not play the song, it's not a electronic thing to play music |
| 10888 |  888 | he did hear music in his cooling glass            | Glass does not produce music.                                                  |
| 20888 |  888 | he did hear music in his cooling glass            | Any sound that might be made by a cooling glass is not music.                  |

Test DataFrame:

|     |   id | FalseSent                                      | reason1                                                  | reason2                                                | reason3                                                            |
|----:|-----:|:-----------------------------------------------|:---------------------------------------------------------|:-------------------------------------------------------|:-------------------------------------------------------------------|
|  76 | 1280 | Beer that is drunk by humans is white          | Beer is made of barley and it is a yellow drink          | A beer that is drunk by humans is not white.           | Beer is brown                                                      |
| 101 |  860 | eating trash food every day makes you stronger | eating trash food every day makes your body fat and weak | eating trash food every day is bad for your health     | Trash food could be contaminated                                   |
| 136 |  777 | he put some cooking oil in his wine            | cooking oil will destroy the taste of the wine           | Cooking oil does not go in wine                        | Cooking oil does not taste nice and therefore would ruin the wine. |
| 174 |  570 | Lobsters live in the mountains                 | Lobsters needs water to live                             | Lobsters live in the sea.                              | Lobsters live in the sea, not the mountains                        |
| 210 | 1929 | the clock shows animals                        | the clock is used to show the time to people             | Clocks are required to tell the time, not show animals | a clock shows the time not animals                                 |
| 235 | 1619 | she put the giraffe in the freezer             | A giraffe is much bigger than the freezer                | There is no way a giraffe is fitting in the freezer.   | A giraffe is too big to be put in a freezer.                       |

In [12]:
def load_data(data_csv, answers_csv, is_test=False):   # 1 Mark

    # Load data_csv
    data = pd.read_csv(data_csv).dropna()

    # Load answers_csv with no header and specified column names
    answers = pd.read_csv(answers_csv, header=None).rename(columns={0: "id", 1: "reason1", 2:"reason2", 3:"reason3"})

    # Merge the data_csv and answers_csv data on 'id' column
    merged_df = pd.merge(data, answers, on='id')

    if is_test:
      return merged_df

    else:
      # Reshape the DataFrame to have a single reason column
      reshaped_df = pd.melt(merged_df, id_vars=['id', 'FalseSent'], value_vars=['reason1', 'reason2', 'reason3'],
                              var_name='reason_type', value_name='reason')

      # Sort by 'id' and 'reason_type'
      reshaped_df.sort_values(['id', 'reason_type'], inplace=True)

      # Drop the 'reason_type' column as it's no longer needed
      reshaped_df.drop(columns=['reason_type'], inplace=True)

      return reshaped_df

In [14]:
train_data_csv = "/content/subtaskC_data_all.csv"
train_answers_csv = "/content/subtaskC_answers_all.csv"
train_data = load_data(train_data_csv, train_answers_csv)
dev_data_csv = "/content/subtaskC_dev_data.csv"
dev_answers_csv = "/content/subtaskC_gold_answers.csv"
dev_data = load_data(dev_data_csv, dev_answers_csv)
test_data_csv = "/content/subtaskC_test_data.csv"
test_answers_csv = "/content/subtaskC_test_gold_answers.csv"
test_data = load_data(test_data_csv, test_answers_csv, True)
if shrink_dataset:
    idxs = train_data["id"].sample(frac=1, random_state=42).unique()[:30]
    train_data = train_data[train_data.id.isin(idxs)]
    idxs = dev_data["id"].sample(frac=1, random_state=42).unique()[:30]
    dev_data = dev_data[dev_data.id.isin(idxs)]
    idxs = test_data["id"].sample(frac=1, random_state=42).unique()[:30]
    test_data = test_data[test_data.id.isin(idxs)]
pd.set_option("display.max_colwidth", None)
print("Train DataFrame:")
display(train_data[:6])
print("Test DataFrame:")
display(test_data[:6])

Train DataFrame:


,id,FalseSent,reason
769,769,Computers is an ingredient used in preparing food,Computers are not used for food and they are not edible
10769,769,Computers is an ingredient used in preparing food,Computer is not something that can be used in preparing food.
20769,769,Computers is an ingredient used in preparing food,You cannot eat a computer
888,888,he did hear music in his cooling glass,"cooling glass can not play the song, it's not a electronic thing to play music"
10888,888,he did hear music in his cooling glass,Glass does not produce music.
20888,888,he did hear music in his cooling glass,Any sound that might be made by a cooling glass is not music.


Test DataFrame:


,id,FalseSent,reason1,reason2,reason3
76,1280,Beer that is drunk by humans is white,Beer is made of barley and it is a yellow drink,A beer that is drunk by humans is not white.,Beer is brown
101,860,eating trash food every day makes you stronger,eating trash food every day makes your body fat and weak,eating trash food every day is bad for your health,Trash food could be contaminated
136,777,he put some cooking oil in his wine,cooking oil will destroy the taste of the wine,Cooking oil does not go in wine,Cooking oil does not taste nice and therefore would ruin the wine.
174,570,Lobsters live in the mountains,Lobsters needs water to live,Lobsters live in the sea.,"Lobsters live in the sea, not the mountains"
210,1929,the clock shows animals,the clock is used to show the time to people,"Clocks are required to tell the time, not show animals",a clock shows the time not animals
235,1619,she put the giraffe in the freezer,A giraffe is much bigger than the freezer,There is no way a giraffe is fitting in the freezer.,A giraffe is too big to be put in a freezer.


In [15]:
train_dataset = Dataset.from_pandas(train_data)
dev_dataset = Dataset.from_pandas(dev_data)
test_dataset = Dataset.from_pandas(test_data)
print("Train Dataset example:")
display(train_dataset[0])
print("Test Dataset example:")
display(test_dataset[0])

Train Dataset example:


{'id': 769,
 'FalseSent': 'Computers is an ingredient used in preparing food',
 'reason': 'Computers are not used for food and they are not edible',
 '__index_level_0__': 769}

Test Dataset example:


{'id': 1280,
 'FalseSent': 'Beer that is drunk by humans is white',
 'reason1': 'Beer is made of barley and it is a yellow drink',
 'reason2': 'A beer that is drunk by humans is not white.',
 'reason3': 'Beer is brown',
 '__index_level_0__': 76}

The `Datasets` should be pre-processed following two different approaches. For the test `Dataset`, you must run the tokenizer on the `FalseSent` column and store the result in the `input_ids` and `attention_mask` fields. For the train and development `Datasets` you must also run the tokenizer on the `reason` column and store the resulting `input_ids` in the `labels` field. In all cases, the tokenizer must pad and truncate the sequences to the `max_length` value.

><pre>
>Train formated Dataset example:
>
>{'id': 769, 'FalseSent': 'Computers is an ingredient used in preparing food', 'reason': 'Computers are not used for food and they are not edible', '__index_level_0__': 769, 'input_ids': [0, 14721, 43990, 16, 41, 16181, 341, 11, 4568, 689, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [0, 14721, 43990, 32, 45, 341, 13, 689, 8, 51, 32, 45, 27532, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
>
>Test formated Dataset example:
>
>{'id': 1280, 'FalseSent': 'Beer that is drunk by humans is white', 'reason1': 'Beer is made of barley and it is a yellow drink', 'reason2': 'A beer that is drunk by humans is not white.', 'reason3': 'Beer is brown', '__index_level_0__': 76, 'input_ids': [0, 45562, 14, 16, 10789, 30, 5868, 16, 1104, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
></pre>

In [18]:
from transformers import BatchEncoding
def preprocess_data(examples, tokenizer, max_length, is_test=False):   # [1 Mark]

    # Tokenize the FalseSent column
    tokenized_false_sent = tokenizer(
        examples["FalseSent"],
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt",
    )

    # Initialize the batch dictionary with input_ids and attention_mask
    batch = {
        "input_ids": tokenized_false_sent["input_ids"],
        "attention_mask": tokenized_false_sent["attention_mask"],
    }

    if not is_test:
        # For train/dev data, tokenize the reason column
        tokenized_reasons = tokenizer(
            examples["reason"],
            padding="max_length",
            truncation=True,
            max_length=max_length,
            return_tensors="pt",
        )
        # Add labels to the batch dictionary
        batch["labels"] = tokenized_reasons["input_ids"]

    # Return the tokenized batch as a BatchEncoding object
    return BatchEncoding(batch)

In [19]:
train_dataset = train_dataset.map(lambda x: preprocess_data(x, tokenizer, max_length), batched=True)
dev_dataset = dev_dataset.map(lambda x: preprocess_data(x, tokenizer, max_length), batched=True)
test_dataset = test_dataset.map(lambda x: preprocess_data(x, tokenizer, max_length, True), batched=True)
print("Train formated Dataset example:\n")
print(train_dataset[0])
print("\nTest formated Dataset example:\n")
print(test_dataset[0])

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Train formated Dataset example:

{'id': 769, 'FalseSent': 'Computers is an ingredient used in preparing food', 'reason': 'Computers are not used for food and they are not edible', '__index_level_0__': 769, 'input_ids': [0, 14721, 43990, 16, 41, 16181, 341, 11, 4568, 689, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [0, 14721, 43990, 32, 45, 341, 13, 689, 8, 51, 32, 45, 27532, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Test formated Dataset example:

{'id': 1280, 'FalseSent': 'Beer that is drunk by humans is white', 'reason1': 'Beer is made of barley and it is a yellow drink', 'reason2': 'A beer that is drunk by humans is not white.', 'reason3': 'Beer is brown', '__index_level_0__': 76, 'input_ids': [0, 45562, 14, 16, 10789, 30, 5868, 16, 1104, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


## Fine-tuning - [5 Marks]

In general, when using a `Trainer` to make predictions, it returns the logits for each class in the task. However, the `Seq2SeqTrainingArguments` class provides an option that allows the `Trainer` to generate sequences of tokens in the prediction. The `create_training_arguments` function must create the `Seq2SeqTrainingArguments` with that option and the hyperparamters passed as arguments. During the training, the model must be evaluated on the development set after every epoch. `TrainingArguments` should include this strategy.

> **Important!** By default, `Trainer` saves a checkpoint of the model every 500 training steps. For this assignment, avoid this behavior by setting `save_strategy="no"` when creating the `TrainingArguments`.

In [27]:
def create_training_arguments(epochs, train_batch_size, learning_rate, output_dir):   # [1 Mark]

    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        num_train_epochs=epochs,
        per_device_train_batch_size=train_batch_size,
        learning_rate=learning_rate,
        evaluation_strategy="epoch",  # Evaluate after every epoch
        save_strategy="no",  # Do not save checkpoints every 500 steps
        predict_with_generate=True,  # Generate sequences during prediction
    )

    return training_args

In [28]:
train_args = create_training_arguments(epochs, train_batch_size, learning_rate, output_dir)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Next, you can create a `Trainer` object initializing the appropriate data collator.

In [29]:
def create_trainer(model, train_args, train_dataset, dev_dataset, tokenizer):   # [1 Mark]

    # Initialize the data collator for Seq2Seq tasks
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    # Create and return the Trainer object
    trainer = Seq2SeqTrainer(
        model=model,
        args=train_args,
        train_dataset=train_dataset,
        eval_dataset=dev_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )
    return trainer

In [30]:
trainer = create_trainer(model, train_args, train_dataset, dev_dataset, tokenizer)

In [31]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: FalseSent, reason, __index_level_0__, id. If FalseSent, reason, __index_level_0__, id are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 90
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 36


Epoch,Training Loss,Validation Loss
1,No log,7.002179
2,No log,6.461193
3,No log,6.248329


The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: FalseSent, reason, __index_level_0__, id. If FalseSent, reason, __index_level_0__, id are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 90
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: FalseSent, reason, __index_level_0__, id. If FalseSent, reason, __index_level_0__, id are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 90
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: FalseSent, reason, __index_level_0__, id. If FalseSent, reason,

TrainOutput(global_step=36, training_loss=7.199645148383246, metrics={'train_runtime': 135.7905, 'train_samples_per_second': 1.988, 'train_steps_per_second': 0.265, 'total_flos': 4019258880000.0, 'train_loss': 7.199645148383246, 'epoch': 3.0})

If you have set the `Seq2SeqTrainingArguments` properly, you could now use the `Trainer` to predict sequences of tokens. Take into account that `Trainer` will return the indexes of the tokens, so the sequence must be decoded to obtain the text strings. The `tokenizer` provides functionality to do this. The result of this process can be stored in the `prediction` column of the test `DataFrame`:

|     |   id | FalseSent                                      | reason1                                                  | reason2                                                | reason3                                                            | prediction                                     |
|----:|-----:|:-----------------------------------------------|:---------------------------------------------------------|:-------------------------------------------------------|:-------------------------------------------------------------------|:-----------------------------------------------|
|  76 | 1280 | Beer that is drunk by humans is white          | Beer is made of barley and it is a yellow drink          | A beer that is drunk by humans is not white.           | Beer is brown                                                      | Beer that is drunk by humans is white                             |
| 101 |  860 | eating trash food every day makes you stronger | eating trash food every day makes your body fat and weak | eating trash food every day is bad for your health     | Trash food could be contaminated                                   | eating trash food every day makes you stronger |
| 136 |  777 | he put some cooking oil in his wine            | cooking oil will destroy the taste of the wine           | Cooking oil does not go in wine                        | Cooking oil does not taste nice and therefore would ruin the wine. | he put some cooking oil in his wine            |
| 174 |  570 | Lobsters live in the mountains                 | Lobsters needs water to live                             | Lobsters live in the sea.                              | Lobsters live in the sea, not the mountains                        | Lobsters live in mountains                 |
| 210 | 1929 | the clock shows animals                        | the clock is used to show the time to people             | Clocks are required to tell the time, not show animals | a clock shows the time not animals                                 | the clock shows animals                        |
| 235 | 1619 | she put the giraffe in the freezer             | A giraffe is much bigger than the freezer                | There is no way a giraffe is fitting in the freezer.   | A giraffe is too big to be put in a freezer.                       | she put the giraffe in the freezer             |


In [32]:
def make_predictions(trainer, test_dataset, tokenizer):   # [1 Mark]

    # Generate predictions
    predictions = trainer.predict(test_dataset)

    # Get the predicted token indices
    predicted_token_indices = predictions.predictions

    # Decode the token indices to text strings
    decoded_predictions = [tokenizer.decode(ids, skip_special_tokens=True) for ids in predicted_token_indices]

    return decoded_predictions

In [33]:
predictions = make_predictions(trainer, test_dataset, tokenizer)
test_data["prediction"] = predictions
test_data

The following columns in the test set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: id, reason3, reason2, reason1, FalseSent, __index_level_0__. If id, reason3, reason2, reason1, FalseSent, __index_level_0__ are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 30
  Batch size = 8


,id,FalseSent,reason1,reason2,reason3,prediction
76,1280,Beer that is drunk by humans is white,Beer is made of barley and it is a yellow drink,A beer that is drunk by humans is not white.,Beer is brown,Beer is not white.
101,860,eating trash food every day makes you stronger,eating trash food every day makes your body fat and weak,eating trash food every day is bad for your health,Trash food could be contaminated,eating trash food every day makes you stronger
136,777,he put some cooking oil in his wine,cooking oil will destroy the taste of the wine,Cooking oil does not go in wine,Cooking oil does not taste nice and therefore would ruin the wine.,he put some cooking oil in his wine
174,570,Lobsters live in the mountains,Lobsters needs water to live,Lobsters live in the sea.,"Lobsters live in the sea, not the mountains",Lobsters live in the mountains
210,1929,the clock shows animals,the clock is used to show the time to people,"Clocks are required to tell the time, not show animals",a clock shows the time not animals,the clock shows animals
235,1619,she put the giraffe in the freezer,A giraffe is much bigger than the freezer,There is no way a giraffe is fitting in the freezer.,A giraffe is too big to be put in a freezer.,she put the giraffe in the freezer
280,979,he installed the carpet on the lake,The carpet will absorb water and sink,Carpets need a subfloor.,A lake would not be able to grip onto carpet.,he installed the carpet on the lake
319,75,My son had us write an essay on The National Monument.,My son isn't smart enough to assign an essay.,My son is studying in the seconds standard only,Children don't ask parents to write essays.,My son had us write an essay on The National Monument.
371,1810,He drove up the stairs to the bedroom,A car is too large to fit upstairs,Stairs are too small and weak for a car to drive up,People don't drive up indoor stairs.,He drove up the stairs to the bedroom
411,774,he put a piece of plastic on the bread,the plastic usually is toxic,You can't eat plastic.,People do not eat plastic because it's not a food,he put a piece of plastic on the bread


The **Subtasks B** of **ComVE** is evaluated using the *bleu* metric. In this assignment, you will also evaluate using *rouge*. With `shrink_dataset` and `base_model` set to `True`, the expected scores are *0.216* and *0.446* for *bleu* and *rouge* respectively. With a full training run, i.e. with `shrink_dataset` and `base_model` set to `False`, the scores should be around *0.228* and *0.461*.

In [36]:
def evaluate_prediction(test_data, metric):   # [2 Marks]

    # Load metric for accuracy evaluation
    accuracy = evaluate.load(metric)

    # Compute the evaluation score based on the metric
    predictions = test_data["prediction"].values
    references = test_data[['reason1', 'reason2', 'reason3']].values
    evaluation_score = accuracy.compute(predictions=predictions, references=references)

    return evaluation_score

In [39]:
evaluate_prediction(test_data, "bleu")

{'bleu': 0.21112609159643486,
 'precisions': [0.5714285714285714,
  0.31213872832369943,
  0.13986013986013987,
  0.07964601769911504],
 'brevity_penalty': 1.0,
 'length_ratio': 1.16,
 'translation_length': 203,
 'reference_length': 175}

In [40]:
evaluate_prediction(test_data, "rouge")

{'rouge1': 0.4610998567422716,
 'rouge2': 0.2491434815041626,
 'rougeL': 0.4410703887329275,
 'rougeLsum': 0.4410075373542866}

The scores for the partial training and the full training are so similar that it would appear that the full training does not provide any benefit in this task. However, it should be noted that the test sets in the two cases are different. More importantly, these results are indicative of the limitations of metrics such as *bleu* and *rouge* for evaluating text generation. Take, for example, the following case from the test set:


| FalseSent                 | reason1                                        | reason2                          | reason3                         |
|:--------------------------|:-----------------------------------------------|:---------------------------------|:--------------------------------|
| Beer that is drunk by humans is white | Beer is made of barley and it is a yellow drink | A beer that is drunk by humans is not white. | Beer is brown |

The predictions obtained by the partial and full trainings and their corresponding scores are the following:

| full training    | prediction                 | bleu     | rouge    |
|:-----------------|:---------------------------|---------:|---------:|
| no               | Beer that is drunk by humans is white  | 0.731    | 0.889    |
| yes              | White beer is not suitable for human consumption. | 0.000    | 0.364    |

The text generated by the full training is a better explanation than the reason generated by the partial training, which is a mere repetition of the nonsensical statement. However, the latter obtains much better scores than the former. Metrics such as *bleu* and *rouge* do not always replace accurately the human judgement.